In [10]:
import requests
from bs4 import BeautifulSoup
import re

# Fetch the page
url = 'https://www.consumerfinance.gov/rules-policy/regulations/1024/17/'
response = requests.get(url)
html_content = response.text

# Parse the HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Extract all paragraphs
paragraphs = soup.find_all('p')
# Combine all text into a single string for easier processing
full_text = '\n'.join([p.get_text() for p in paragraphs])

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1250, chunk_overlap=20)

chunks = text_splitter.split_text(full_text)


In [19]:
print("Number of chunks: ", len(chunks))
print("Chunk 1: ", chunks[0])
print("Chunk 2: ", chunks[1])

Number of chunks:  41
Chunk 1:  (a) General. This section sets out the requirements for an escrow account that a lender establishes in connection with a federally related mortgage loan. It sets limits for escrow accounts using calculations based on monthly payments and disbursements within a calendar year. If an escrow account involves biweekly or any other payment period, the requirements in this section shall be modified accordingly. A Public Guidance Document entitled “Biweekly Payments - Example” provides examples of biweekly accounting and a Public Guidance Document entitled “Annual Escrow Account Disclosure Statement - Example” provides examples of a 3-year accounting cycle that may be used in accordance with paragraph (c)(9) of this section. A Public Guidance Document entitled “Consumer Disclosure for Voluntary Escrow Account Payments” provides a model disclosure format that originators and servicers are encouraged, but not required, to provide to consumers when the originator o

In [34]:
from openai import OpenAI
from utility.utils import get_openai_api_key

OpenAI.api_key = get_openai_api_key()
client = OpenAI()



chat_completion = client.chat.completions.create(
    messages=[
        {
          "role":"system",
          "content": """
          # Role
          you are a worldclass llm training data generator. 
          
          # Task
          You are generating q/a pairs for finetuning our own llm model.
          create a json of questions and answers. 
          
          # Specifics
          Context relecance for the Q/A is really important, else our business will loose money on computation resources 
          
          # Notes
          Make sure to generate JSON format for the Q/A pairs
          
          """  
        },
        {
            "role": "user",
            "content": "(a) General. This section sets out the requirements for an escrow account that a lender establishes in connection with a federally related mortgage loan. It sets limits for escrow accounts using calculations based on monthly payments and disbursements within a calendar year. If an escrow account involves biweekly or any other payment period, the requirements in this section shall be modified accordingly. A Public Guidance Document entitled “Biweekly Payments - Example” provides examples of biweekly accounting and a Public Guidance Document entitled “Annual Escrow Account Disclosure Statement - Example” provides examples of a 3-year accounting cycle that may be used in accordance with paragraph (c)(9) of this section. A Public Guidance Document entitled “Consumer Disclosure for Voluntary Escrow Account Payments” provides a model disclosure format that originators and servicers are encouraged, but not required, to provide to consumers when the originator or servicer anticipates a substantial increase in disbursements from the escrow account after the first year of the loan. The disclosures in that model format may be combined with or included in the Initial Escrow Account Statement required in § 1024.17(g).",
        }
    ],
    model="gpt-3.5-turbo",
    response_format={ "type": "json_object" }
)
print(chat_completion.choices[0].message.content)

{
  "questions": [
    {
      "question": "What does this section specify about the requirements for an escrow account established by a lender in connection with a federally related mortgage loan?",
      "answer": "This section sets out the requirements for an escrow account established by a lender in connection with a federally related mortgage loan. It establishes limits for escrow accounts based on monthly payments and disbursements within a calendar year. The requirements are adjusted for escrow accounts involving payment periods other than monthly, such as biweekly."
    },
    {
      "question": "How are the limits for escrow accounts determined under this section?",
      "answer": "The limits for escrow accounts are determined based on calculations involving monthly payments and disbursements within a calendar year. Special modifications are made for escrow accounts with payment periods different from monthly, as detailed in the section."
    },
    {
      "question": "What